In [1]:
import pandas as pd
import os
import re
import torch
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

labeled_data_path = os.path.join(DATA_DIR, "cleaning_step1")

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# 모델과 토크나이저 로드
model_name = 'Saxo/Linkbricks-Horizon-AI-Korean-Pro-8B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(DEVICE)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm)

In [3]:
df_list = []
for label in range(7) :
    df = pd.read_csv(labeled_data_path+f"/c1_label_dropped_{label}.csv")
    df_list.append(df)
    print(df.head())

                    ID                                               text  \
0  ynat-v1_train_00015                     KISA 박민정 책임연구원이 APTLD 이사로 선출되다   
1  ynat-v1_train_00066  UEFA 챔피언스리그 4강 마드리드 더비, 레알 마드리드와 아틀레티코 마드리드가 맞붙는다   
2  ynat-v1_train_00079                          본트카 애스턴마틴 스파 국내 4백500대 생산   
3  ynat-v1_train_00087                               수능 6개 시험장 울릉도서 한 화장실   
4  ynat-v1_train_00113                             신간 "지적자본주의와 사회적 기업" 출간   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  
                    ID                                text  target
0  ynat-v1_train_00020  추신수 타율 0.265로 시즌 마감, 최지만은 19홈런 6타점       1
1  ynat-v1_train_00021        KIA, 수단 팬들에게 호평받는 K4 M족 한 마리       1
2  ynat-v1_train_00028         KFA, 2023년 7월 70회 월드컵 대회 개최       1
3  ynat-v1_train_00029           축구 마스터즈, 인천에서 35라운드 경기 진행       1
4  ynat-v1_train_00032           박항 감독의 베트남 축구, 한국과의 경기 준비       1
                    ID                              text  targe

## 1. Paraphrasing

In [ ]:
for idx, df in enumerate(df_list) :
    temp_df = df.copy()
    for tidx, noised_text in enumerate(df['text']) :
        prompt = f"""
                    당신은 전문적인 한국어 신문 헤드라인 교정 전문가입니다. 주어진 헤드라인의 오류를 최소한으로 수정하고, 원문의 의도와 핵심 내용을 최대한 유지해야 합니다. 다음 지침을 엄격히 따라주세요:

                    1. 원문 분석: 헤드라인의 주요 키워드와 전체적인 의미를 파악합니다.
                    2. 최소 수정: 명백한 오류만을 수정하고, 불필요한 변경은 피합니다.
                    3. 맥락 유지: 원문의 의도와 문맥을 최대한 유지합니다.
                    4. 오타 수정: 명확한 오타만 수정하고, 의심스러운 경우 원문을 유지합니다.
                    5. 문법 교정: 문법적 오류가 명확한 경우에만 수정합니다.
                    6. 불필요한 추가 금지: 원문에 없는 정보를 임의로 추가하지 않습니다.
                    7. 확신 없는 경우 유지: 교정이 확실하지 않은 경우, 원문을 그대로 유지합니다.

                    예시:
                    1. 원문: 문인 당2, 4동 민관2동 7사위 철거
                    교정: 문인동 2, 4동과 민관동 2동 7층 아파트 철거

                    2. 원문: WEF 일련의 이벤트에서 창조경제 논의
                    교정: WEF, 일련의 이벤트에서 창조경제 논의

                    3. 원문: 김종길 리먼 사태처럼 충격적 인 6Mq
                    교정: 김종길 "리먼 사태처럼 충격적인 상황 우려"

                    이제 다음 헤드라인을 위의 지침과 예시를 참고하여 최소한으로 교정해주세요. 확실하지 않은 부분은 원문 그대로 유지하세요:

                    원문: {noised_text}
                    교정:"""


        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=70,
                num_return_sequences=1,
                temperature=0.8,
                top_p=0.95,
                do_sample=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_text = generated_text.split("교정:")[-1].split('\n')[0]
        
        temp_df.loc[tidx, 'text'] = cleaned_text
        print("origin : ", noised_text)
        print("cleaned : ", cleaned_text)

    temp_df.to_csv(f'./data/cleaning_pp/cleaned_label_{idx}.csv', index=False)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  KISA 박민정 책임연구원이 APTLD 이사로 선출되다
cleaned :   KISA 박민정 책임연구원이 APTLD 이사로 선출되다


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  UEFA 챔피언스리그 4강 마드리드 더비, 레알 마드리드와 아틀레티코 마드리드가 맞붙는다
cleaned :   UEFA 챔피언스리그 4강 마드리드 더비, 레알 마드리드와 아틀레티코 마드리드가 맞붙는다


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

origin :  폭풍으로 갇힌 R주 산마을, 지금까지 이 내린 눈의 양은
cleaned :   폭풍으로 갇힌 R주 산마을, 지금까지 이 내린


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  간 기와 외상 3차
cleaned :   간 기와 외상 3차


KeyboardInterrupt: 

: 